# **OpenPCDet**

**Author:** [Yi-Jie Wong](https://www.linkedin.com/in/wongyijie/) & [Wingates Voon](https://www.linkedin.com/in/wingates-voon-5858391a0/) & [Yan-Chai Hum](https://www2.utar.edu.my/cv/index.jsp?cv=humyc&reqPageId=aboutMe)<br>
**GitHub:** [OpenPCDet](https://github.com/yjwong1999/OpenPCDet.git)<br>
**Date created:** 2023/05/07<br>
**Last modified:** 2023/10/30<br>
**Description:** Training 3D Object Detection Models using a highly specialized OpenPCDet-based pipeline

Pipeline Definition
1.   **Preprocessing** (standard: downsampling + removal of outlier point clouds)
2.   **Custon Preprocessing** (splitting point clouds into 2 + cropping ROI)
3.   **Annotation** (using LabelCLoud, which is a Python based tool)
4.   **Automated Augmentation** (an augmentation pipeline we devised for the project)
5.   **Convert raw data to OpenPCDet format** (necessary conversion)
6.   **OpenPCDet's own processing** (which is included in Part 7./8.)
7.   **OpenPCDet training**
8.   **OpenPCDet deployment**

### Get ready the environment and dependencies

In [1]:
# check CUDA version (11.8 in this case)
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
# check torch cuda version, make sure same as the version provided by nvcc
import torch
torch.__version__

'2.1.0+cu118'

In [3]:
# install the spconv with the same version as pytorch and nvcc
# for this case, we need to install spconv for cuda 11.8 version
# hence, install spconv-cu118
!pip install spconv-cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 16.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116933 sha256=229b0521198a28f43615c90eddb9defe7f202aab3513119d550ce1c4dbb8694f
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [5]:
# git clone the private OpenPCDet
# to git clone a private repo, pls follow the link: https://stackoverflow.com/a/51441990
# you will need personal access token to install, you can follow: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#creating-a-personal-access-token-classic
!git clone https://<personal access token>@github.com/wingatesv/OpenPCDet.git

Cloning into 'OpenPCDet'...
remote: Enumerating objects: 3971, done.
remote: Counting objects: 100% (3971/3971), done.
remote: Compressing objects: 100% (1147/1147), done.
remote: Total 3971 (delta 2767), reused 3971 (delta 2767), pack-reused 0
Receiving objects: 100% (3971/3971), 2.91 MiB | 8.60 MiB/s, done.
Resolving deltas: 100% (2767/2767), done.


In [6]:
%cd OpenPCDet

/content/OpenPCDet


In [6]:
# this will authomatically setup the OpenPCDet
!python setup.py develop

running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.



In [7]:
# install remaining dependencies following the version specified

from IPython.display import clear_output
# !pip install pandas==1.5.3
!pip install plyfile==1.0.1
# !pip install opencv-python==4.7.0.72
!pip install av2==0.2.1
!pip install kornia==0.5.8
!pip install mayavi==4.8.1
!pip install PyQt5==5.15.9
!pip install open3d==0.17.0

clear_output()

### Get data (follow Option 2 for simplicity's sake)

In [8]:
# Option 1: Process data from top to bottom (data & label collection -> preprocessing -> augmentation -> conversion to OpenPCDet format)
# This .sh file will do Step 1 to Step 5
# Step 1: Preprocessing (standard: downsampling + removal of outlier point clouds)
# Step 2: Custom Preprocessing (splitting point clouds into 2 + cropping ROI)
# Step 3: Annotation using LabelCLoud (for simplicity sake, we assumed this is ready)
# Step 4: Automated Augmentation (an augmentation pipeline we devised for the project)
# Step 5: Convert raw data to OpenPCDet format (necessary conversion)

# !bash bash_data.sh

In [8]:
# Option 2: To start after augmentation (conversion to OpenPCDet format))

!bash bash_ready_data.sh

bash_ready_data.sh: line 15: /root/anaconda3/bin/activate: No such file or directory
bash_ready_data.sh: line 16: conda: command not found
 
bash_ready_data.sh: line 18: conda: command not found
Created directory data_raw

bash_ready_data.sh: line 42: https://drive.google.com/file/d/1Q9Vy_Gd3OLKXVJ9LO0KatQIfu_sPW3Hs/view?usp=sharing: No such file or directory
The client data zip file does not exists. Downloading now...

--2023-11-27 08:24:00--  https://docs.google.com/uc?export=download&confirm=t&id=1PEhDVrk0rn1fxMpa_M6TVRzCeG0_x4pT
Resolving docs.google.com (docs.google.com)... 74.125.141.138, 74.125.141.113, 74.125.141.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.141.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/adtnrpbahu7fcra472r7bedmjf4fbna9/1701073425000/18333994960236708988/*/1PEhDVrk0rn1fxMpa_M6TVRzCeG0_x4pT?e=download&uuid=c748

### Start Training

In [10]:
# Option 1: Train PointPillar via conventional method
# Step 6. OpenPCDet's own processing (which is included in Part 7./8.)
# Step 7. OpenPCDet training

!bash bash_train.sh

bash_train.sh: line 16: /root/anaconda3/bin/activate: No such file or directory
bash_train.sh: line 17: conda: command not found
 
bash_train.sh: line 19: conda: command not found
Raw data + label -> OpenPCDet standardized format (in data/custom)...: 100% 1362/1362 [02:50<00:00,  7.98it/s]
[-1.82, -0.41, 0.06, 27.7, 20.24, 40.72]
[-1.82, -0.41, 0.06, 30.18, 31.59, 40.72]

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'pcdet.datasets.custom.custom_dataset' found in sys.modules after import of package 'pcdet.datasets.custom', but prior to execution of 'pcdet.datasets.custom.custom_dataset'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-11-25 15:11:36,469   INFO  Loading Custom dataset.
2023-11-25 15:11:36,470   INFO  Total samples for CUSTOM dataset: 0
------------------------Start to generate data infos------------------------
train sample_idx: augmented_000 RF 218_R10C36_F_Snap3D_part2
train sample_idx: RF 057_R2C34_F_Snap3D_part1
train sample_idx: aug

In [9]:
# Option 2: Train PointPillar via conventional method
# Step 6. OpenPCDet's own processing (which is included in Part 7./8.)
# Step 7. OpenPCDet training via Reptile

# Disclaimer
# Please note that, we cant finish the reptile training in Colab due to limited runtime
# Hence, we only use 10 epochs for each training (we used 100 epochs for actual training)

!bash bash_reptile.sh

Streaming output truncated to the last 5000 lines.
gt_database sample: 256/685
gt_database sample: 257/685
gt_database sample: 258/685
gt_database sample: 259/685
gt_database sample: 260/685
gt_database sample: 261/685
gt_database sample: 262/685
gt_database sample: 263/685
gt_database sample: 264/685
gt_database sample: 265/685
gt_database sample: 266/685
gt_database sample: 267/685
gt_database sample: 268/685
gt_database sample: 269/685
gt_database sample: 270/685
gt_database sample: 271/685
gt_database sample: 272/685
gt_database sample: 273/685
gt_database sample: 274/685
gt_database sample: 275/685
gt_database sample: 276/685
gt_database sample: 277/685
gt_database sample: 278/685
gt_database sample: 279/685
gt_database sample: 280/685
gt_database sample: 281/685
gt_database sample: 282/685
gt_database sample: 283/685
gt_database sample: 284/685
gt_database sample: 285/685
gt_database sample: 286/685
gt_database sample: 287/685
gt_database sample: 288/685
gt_database sample: 289/6